In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Assessment_4').getOrCreate()

In [3]:
data = spark.read.csv(path='/content/used_car_sales.csv',header=True,inferSchema=True)

In [4]:
data.show(3)

+------+----------------+--------------+--------+-----------------+---------+-----+---------+---------------+---------------+------+-----------------+-------+----------+---------------+--------------+---------------+----------+-----------------+------------+--------+----------------+------------+------------------+--------+
|    ID|Distributor Name|      Location|Car Name|Manufacturer Name| Car Type|Color|  Gearbox|Number of Seats|Number of Doors|Energy|Manufactured Year|Price-$|Mileage-KM|Engine Power-HP|Purchased Date|Car Sale Status| Sold Date|Purchased Price-$|Sold Price-$|Margin-%|Sales Agent Name|Sales Rating|Sales Commission-$|Feedback|
+------+----------------+--------------+--------+-----------------+---------+-----+---------+---------------+---------------+------+-----------------+-------+----------+---------------+--------------+---------------+----------+-----------------+------------+--------+----------------+------------+------------------+--------+
|O2KE18|         Carmu

In [5]:
data.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Distributor Name: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Car Name: string (nullable = true)
 |-- Manufacturer Name: string (nullable = true)
 |-- Car Type: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Gearbox: string (nullable = true)
 |-- Number of Seats: integer (nullable = true)
 |-- Number of Doors: integer (nullable = true)
 |-- Energy: string (nullable = true)
 |-- Manufactured Year: integer (nullable = true)
 |-- Price-$: integer (nullable = true)
 |-- Mileage-KM: integer (nullable = true)
 |-- Engine Power-HP: integer (nullable = true)
 |-- Purchased Date: string (nullable = true)
 |-- Car Sale Status: string (nullable = true)
 |-- Sold Date: string (nullable = true)
 |-- Purchased Price-$: integer (nullable = true)
 |-- Sold Price-$: integer (nullable = true)
 |-- Margin-%: integer (nullable = true)
 |-- Sales Agent Name: string (nullable = true)
 |-- Sales Rating: integer (nullabl

In [6]:
from pyspark.sql.functions import col,replace

data = data.select([col(new_col).alias(new_col.replace(" ","_")) for new_col in data.columns])

In [7]:
data.show(5)

+------+----------------+--------------+--------+-----------------+-----------+-----+---------+---------------+---------------+--------+-----------------+-------+----------+---------------+--------------+---------------+----------+-----------------+------------+--------+----------------+------------+------------------+--------+
|    ID|Distributor_Name|      Location|Car_Name|Manufacturer_Name|   Car_Type|Color|  Gearbox|Number_of_Seats|Number_of_Doors|  Energy|Manufactured_Year|Price-$|Mileage-KM|Engine_Power-HP|Purchased_Date|Car_Sale_Status| Sold_Date|Purchased_Price-$|Sold_Price-$|Margin-%|Sales_Agent_Name|Sales_Rating|Sales_Commission-$|Feedback|
+------+----------------+--------------+--------+-----------------+-----------+-----+---------+---------------+---------------+--------+-----------------+-------+----------+---------------+--------------+---------------+----------+-----------------+------------+--------+----------------+------------+------------------+--------+
|O2KE18|  

In [8]:
# as alphanumeric string is difficult for joining any other operation we'll add surrogate key

from pyspark.sql.functions import monotonically_increasing_id

data = data.withColumn('new_id',monotonically_increasing_id()+1)

In [9]:
from pyspark.sql.functions import isnull

null_check = [col(check).isNull() for check in data.columns]

In [10]:
null_check

[Column<'(ID IS NULL)'>,
 Column<'(Distributor_Name IS NULL)'>,
 Column<'(Location IS NULL)'>,
 Column<'(Car_Name IS NULL)'>,
 Column<'(Manufacturer_Name IS NULL)'>,
 Column<'(Car_Type IS NULL)'>,
 Column<'(Color IS NULL)'>,
 Column<'(Gearbox IS NULL)'>,
 Column<'(Number_of_Seats IS NULL)'>,
 Column<'(Number_of_Doors IS NULL)'>,
 Column<'(Energy IS NULL)'>,
 Column<'(Manufactured_Year IS NULL)'>,
 Column<'(Price-$ IS NULL)'>,
 Column<'(Mileage-KM IS NULL)'>,
 Column<'(Engine_Power-HP IS NULL)'>,
 Column<'(Purchased_Date IS NULL)'>,
 Column<'(Car_Sale_Status IS NULL)'>,
 Column<'(Sold_Date IS NULL)'>,
 Column<'(Purchased_Price-$ IS NULL)'>,
 Column<'(Sold_Price-$ IS NULL)'>,
 Column<'(Margin-% IS NULL)'>,
 Column<'(Sales_Agent_Name IS NULL)'>,
 Column<'(Sales_Rating IS NULL)'>,
 Column<'(Sales_Commission-$ IS NULL)'>,
 Column<'(Feedback IS NULL)'>,
 Column<'(new_id IS NULL)'>]

In [11]:
from pyspark.sql.functions import sum,isnull

null_counts = data.select([sum(col(c).isNull().cast("int")).alias(c) for c in data.columns])


null_counts.show()

+---+----------------+--------+--------+-----------------+--------+-----+-------+---------------+---------------+------+-----------------+-------+----------+---------------+--------------+---------------+---------+-----------------+------------+--------+----------------+------------+------------------+--------+------+
| ID|Distributor_Name|Location|Car_Name|Manufacturer_Name|Car_Type|Color|Gearbox|Number_of_Seats|Number_of_Doors|Energy|Manufactured_Year|Price-$|Mileage-KM|Engine_Power-HP|Purchased_Date|Car_Sale_Status|Sold_Date|Purchased_Price-$|Sold_Price-$|Margin-%|Sales_Agent_Name|Sales_Rating|Sales_Commission-$|Feedback|new_id|
+---+----------------+--------+--------+-----------------+--------+-----+-------+---------------+---------------+------+-----------------+-------+----------+---------------+--------------+---------------+---------+-----------------+------------+--------+----------------+------------+------------------+--------+------+
|  0|               0|       0|       0|

In [12]:
# now adding this surrogate key to the dimension tables

car_dimension_dataset = data.select('new_ID',
                                    'ID',
                                    'Car_Name',

                                    'Manufacturer_Name',
                                    'Car_Type',
                                    'Color')


car_dimension_dataset.show(4)

+------+------+--------+-----------------+-----------+-----+
|new_ID|    ID|Car_Name|Manufacturer_Name|   Car_Type|Color|
+------+------+--------+-----------------+-----------+-----+
|     1|O2KE18|Fortuner|           Toyota|        SUV| Gray|
|     2|EPMPC8|   Creta|          Hyundai|  Hatchback| Blue|
|     3|SQKXAP| Scorpio|         Mahindra|        SUV| Gray|
|     4|PWP2QK|   Plato|            Prazo|Convertible| Gray|
+------+------+--------+-----------------+-----------+-----+
only showing top 4 rows



In [13]:
distributor_dimension = data.select('new_ID',
                                    'ID',
                                    'Distributor_Name',
                                    'Location')

distributor_dimension.show(4)

+------+------+----------------+--------------+
|new_ID|    ID|Distributor_Name|      Location|
+------+------+----------------+--------------+
|     1|O2KE18|         Carmudi|    California|
|     2|EPMPC8|       Carousell|  Philadelphia|
|     3|SQKXAP|         Carsome|North Carolina|
|     4|PWP2QK|         Trivett|North Carolina|
+------+------+----------------+--------------+
only showing top 4 rows



**SYNTAX for JOIN**

```
result_df = df1.join(df2, df1["common_column"] == df2["common_column"], "join_type")

```



In [14]:
joined_new_data = car_dimension_dataset.join(distributor_dimension,car_dimension_dataset['new_ID']==distributor_dimension['new_ID'],"Inner")

joined_new_data.show(4)

+------+------+--------+-----------------+-----------+-----+------+------+----------------+--------------+
|new_ID|    ID|Car_Name|Manufacturer_Name|   Car_Type|Color|new_ID|    ID|Distributor_Name|      Location|
+------+------+--------+-----------------+-----------+-----+------+------+----------------+--------------+
|     1|O2KE18|Fortuner|           Toyota|        SUV| Gray|     1|O2KE18|         Carmudi|    California|
|     2|EPMPC8|   Creta|          Hyundai|  Hatchback| Blue|     2|EPMPC8|       Carousell|  Philadelphia|
|     3|SQKXAP| Scorpio|         Mahindra|        SUV| Gray|     3|SQKXAP|         Carsome|North Carolina|
|     4|PWP2QK|   Plato|            Prazo|Convertible| Gray|     4|PWP2QK|         Trivett|North Carolina|
+------+------+--------+-----------------+-----------+-----+------+------+----------------+--------------+
only showing top 4 rows



In [15]:
joined_data = car_dimension_dataset.alias('car').join(distributor_dimension.alias('Dis'), on='new_ID',how='inner').\
select(col('car.new_ID'),
        col('car.ID'),
        col('car.Car_Name'),
        col('car.Car_Type'),
        col('car.Manufacturer_Name'),
        col('car.Color'),
        col('Dis.Distributor_Name'),
        col('Dis.Location'))

In [16]:
joined_data.show(4)

+------+------+--------+-----------+-----------------+-----+----------------+--------------+
|new_ID|    ID|Car_Name|   Car_Type|Manufacturer_Name|Color|Distributor_Name|      Location|
+------+------+--------+-----------+-----------------+-----+----------------+--------------+
|     1|O2KE18|Fortuner|        SUV|           Toyota| Gray|         Carmudi|    California|
|     2|EPMPC8|   Creta|  Hatchback|          Hyundai| Blue|       Carousell|  Philadelphia|
|     3|SQKXAP| Scorpio|        SUV|         Mahindra| Gray|         Carsome|North Carolina|
|     4|PWP2QK|   Plato|Convertible|            Prazo| Gray|         Trivett|North Carolina|
+------+------+--------+-----------+-----------------+-----+----------------+--------------+
only showing top 4 rows



But the first JOIN method is much effective when:



*   **Column names differ in the two DataFrames (e.g., df1["new_ID"] and df2["new_ID"]).**
*  **You need to join on multiple conditions (e.g., df1["ID"] == df2["ID"] & df1["Name"] == df2["Name"]).**
*  **The common column is not named the same in both DataFrames.**

Other wise 2nd Join is much faster


In [17]:
joined_data.write.parquet(path="/content/transformed_data.parquet")

In [18]:
external_data =  [
    (1, "Toyota Corolla", "Sedan", "Toyota", "Red", 20000),
    (2, "Honda Civic", None, "Honda", "Blue", 22000),  # Null in Car_Type
    (3, None, "SUV", "Ford", "Black", 25000),          # Null in Car_Name
    (4, "Tesla Model 3", "Sedan", "Tesla", None, 40000), # Null in Color
    (5, "BMW X5", "SUV", "BMW", "White", None)         # Null in Price
]

In [19]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [24]:
schema = StructType([StructField("ID",IntegerType()),
                     StructField("Car_Name",StringType()),
                     StructField('Model',StringType()),
                     StructField("Company",StringType()),
                     StructField("Color",StringType()),
                     StructField("Price($)",IntegerType())])

In [25]:
df = spark.createDataFrame(data=external_data,schema=schema)

In [39]:
df.show()

+---+--------------+-----+-------+-----+--------+
| ID|      Car_Name|Model|Company|Color|Price($)|
+---+--------------+-----+-------+-----+--------+
|  1|Toyota Corolla|Sedan| Toyota|  Red|   20000|
|  2|   Honda Civic| NULL|  Honda| Blue|   22000|
|  3|          NULL|  SUV|   Ford|Black|   25000|
|  4| Tesla Model 3|Sedan|  Tesla| NULL|   40000|
|  5|        BMW X5|  SUV|    BMW|White|    NULL|
+---+--------------+-----+-------+-----+--------+



In [26]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Car_Name: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Price($): integer (nullable = true)





```
df_filled = df.fillna({"column_name1": value1, "column_name2": value2})

```



Since PySpark does not have a median() function, we use approxQuantile(), which is efficient for large datasets.

approxQuantile() is fast and works on big data.



```
# Compute Median (50th percentile)
median_value = df.approxQuantile("Price($)", [0.5], 0.01)[0]  # 0.5 = median

```



In [38]:
# from pyspark.sql.functions import percentile_approx,col,expr

# median_price = df.select(expr('median(col("Price($)"))'))
# median_price

AnalysisException: [UNRESOLVED_ROUTINE] Cannot resolve function `col` on search path [`system`.`builtin`, `system`.`session`, `spark_catalog`.`default`].; line 1 pos 7

In [40]:
df1 = df.fillna({"Price($)":-1})

In [42]:
df1.show()

+---+--------------+-----+-------+-----+--------+
| ID|      Car_Name|Model|Company|Color|Price($)|
+---+--------------+-----+-------+-----+--------+
|  1|Toyota Corolla|Sedan| Toyota|  Red|   20000|
|  2|   Honda Civic| NULL|  Honda| Blue|   22000|
|  3|          NULL|  SUV|   Ford|Black|   25000|
|  4| Tesla Model 3|Sedan|  Tesla| NULL|   40000|
|  5|        BMW X5|  SUV|    BMW|White|      -1|
+---+--------------+-----+-------+-----+--------+



In [46]:
df2 = df.dropna(how = 'any',subset = ['Car_Name']) # here 'any' means if any one value is null then the row gets dropped

In [47]:
df2.show()

+---+--------------+-----+-------+-----+--------+
| ID|      Car_Name|Model|Company|Color|Price($)|
+---+--------------+-----+-------+-----+--------+
|  1|Toyota Corolla|Sedan| Toyota|  Red|   20000|
|  2|   Honda Civic| NULL|  Honda| Blue|   22000|
|  4| Tesla Model 3|Sedan|  Tesla| NULL|   40000|
|  5|        BMW X5|  SUV|    BMW|White|    NULL|
+---+--------------+-----+-------+-----+--------+



In [55]:
df3 = df.dropna(how='any',subset=['Car_Name','Model','Color']) # 'all' means all the values must be null inorder to get removed

In [56]:
df3.show()

+---+--------------+-----+-------+-----+--------+
| ID|      Car_Name|Model|Company|Color|Price($)|
+---+--------------+-----+-------+-----+--------+
|  1|Toyota Corolla|Sedan| Toyota|  Red|   20000|
|  5|        BMW X5|  SUV|    BMW|White|    NULL|
+---+--------------+-----+-------+-----+--------+

